# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split


[nltk_data] Downloading package punkt to /home/brunom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/brunom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# load data from database
engine = create_engine('sqlite:///data/disasterResponse.db')
df = pd.read_sql('SELECT * FROM messages_categories', engine)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [35]:
stemmer = PorterStemmer()
def tokenize(sentence):
 
    return [stemmer.stem(word) for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

In [36]:
tokenize(X[2])

['look', 'someon', 'name']